In [1]:
# Setup
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from aac_crud_module import AnimalShelter  # CRUD Module

# Instantiate AnimalShelter object with credentials
shelter = AnimalShelter()

# Pull all animal data into a DataFrame
df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'], inplace=True)  # remove ObjectId

# App Setup
app = JupyterDash("GraziosoDashboard")

# Layout
app.layout = html.Div([
    html.Center([
        html.Img(src='assets/Grazioso Salvare Logo.png', style={'height': '150px'}),
        html.H1("Grazioso Salvare Animal Dashboard", style={'color': '#B22222'}),
        html.H3("Developed by Tony McCormack", style={'color': '#555'}),
    ]),
    html.Hr(),

    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block', 'marginRight': '20px'}
        )
    ], style={'padding': '20px'}),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '120px', 'width': '120px', 'maxWidth': '180px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis'
        },
        row_selectable='single',
        selected_rows=[0]
    ),

    html.Br(),
    html.Div(id='map-id'),
])

# Callback to filter table based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def filter_animals(rescue_type):
    if rescue_type == 'Water Rescue':
        query = {"animal_type": "Dog", "breed": {"$regex": "Labrador Retriever|Chesapeake Bay Retriever|Newfoundland", "$options": "i"}}
    elif rescue_type == 'Mountain or Wilderness Rescue':
        query = {"animal_type": "Dog", "breed": {"$regex": "German Shepherd|Alaskan Malamute|Old English Sheepdog|Siberian Husky|Rottweiler", "$options": "i"}}
    elif rescue_type == 'Disaster or Individual Tracking':
        query = {"animal_type": "Dog", "breed": {"$regex": "Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler", "$options": "i"}}
    else:
        query = {}  # Reset
    
    filtered = pd.DataFrame.from_records(shelter.read(query))
    if '_id' in filtered.columns:
        filtered.drop(columns=['_id'], inplace=True)
    return filtered.to_dict('records')

# Callback for map update based on selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row]['location_lat'], dff.iloc[row]['location_long']],
                      children=[
                          dl.Tooltip(dff.iloc[row]['breed']),
                          dl.Popup([
                              html.H1("Animal Name"),
                              html.P(dff.iloc[row]['name'])
                          ])
                      ])
        ])
    ]

# Run app
app.run_server(mode='inline')